# FRB example

## v2 -- Using PATH object

In [1]:
# imports
from importlib import reload
import os
from pkg_resources import resource_filename

import numpy as np

import pandas

from astropy.coordinates import SkyCoord
#from astropy.io import fits

from astropath import path
from astropath import localization

## Dummy FRB (based on 180924)

## Instantiate PATH

In [2]:
Path = path.PATH()

## Localization

## Here is the full data model for the localization options

In [3]:
localization.localization_dmodel

{'type': {'dtype': str,
  'options': ['eellipse', 'wcs', 'healpix'],
  'help': 'Localization type.'},
 'center_coord': {'dtype': astropy.coordinates.sky_coordinate.SkyCoord,
  'help': '"Central" coordinate'},
 'eellipse': {'dtype': dict,
  'help': 'Error ellipse with keys "a [arcsec]", "b [arcsec]", "theta [deg]"'},
 'healpix_data': {'dtype': (numpy.ndarray, astropy.table.table.Table),
  'help': 'Data containing the healpix information. Input either as a simple numpy array for a full NESTED array or an astropy Table for NUNIQ format with columns UNIQ and PROBDENSITY.'},
 'healpix_nside': {'dtype': int, 'help': 'NSIDE value of healpix map.'},
 'healpix_ordering': {'dtype': str,
  'options': ['NESTED', 'NUNIQ'],
  'help': 'Ordering of the healpix information.'},
 'healpix_coord': {'dtype': str,
  'options': ['C'],
  'help': 'Coordinate system of the healpix.'},
 'wcs_data': {'dtype': numpy.ndarray, 'help': 'PDF of localization.'},
 'wcs_WCS': {'dtype': astropy.wcs.wcs.WCS, 'help': 'WCS o

## We will use the Error Ellipse type

In [4]:
## FRB Coord
frb_coord = SkyCoord('21h44m25.255s -40d54m00.10s', frame='icrs')
frb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (326.10522917, -40.90002778)>

## Error ellipse

In [5]:
#eellipse = dict(a=0.3, b=0.1, theta=45.)
#eellipse = dict(a=3, b=3, theta=90.)
eellipse = dict(a=0.1, b=0.1, theta=0.)

### Set it

In [6]:
Path.init_localization('eellipse', center_coord=frb_coord, eellipse=eellipse)

In [7]:
Path.localiz

{'type': 'eellipse',
 'center_coord': <SkyCoord (ICRS): (ra, dec) in deg
     (326.10522917, -40.90002778)>,
 'eellipse': {'a': 0.1, 'b': 0.1, 'theta': 0.0}}

## Candidates

## Load

In [8]:
cand_file = os.path.join(resource_filename('astropath', 'data'), 'frb_example', 'frb180924_candidates.csv')

In [9]:
candidates = pandas.read_csv(cand_file, index_col=0)
candidates.head()

,ra,dec,half_light,VLT_FORS2_g
3,326.101738,-40.899771,0.458869,25.295416
7,326.105365,-40.900239,1.308629,21.319569
8,326.104186,-40.900180,0.814683,24.272838
9,326.106237,-40.899279,0.501247,25.474828


## Init in PATH

In [10]:
Path.init_candidates(candidates.ra.values,
                     candidates.dec.values,
                     candidates.half_light.values,
                     mag=candidates.VLT_FORS2_g.values)

In [11]:
Path.candidates

,ra,dec,ang_size,mag
0,326.101738,-40.899771,0.458869,25.295416
1,326.105365,-40.900239,1.308629,21.319569
2,326.104186,-40.900180,0.814683,24.272838
3,326.106237,-40.899279,0.501247,25.474828


## Priors

## Candidate prior

In [12]:
Path.init_cand_prior('inverse', P_U=0.)

## Offset prior

In [13]:
Path.init_theta_prior('exp', 6.)

## Calculate

In [14]:
Path.calc_priors()

array([0.0316325 , 0.87225473, 0.06831713, 0.02779563])

In [15]:
Path.candidates

,ra,dec,ang_size,mag,P_O
0,326.101738,-40.899771,0.458869,25.295416,0.031633
1,326.105365,-40.900239,1.308629,21.319569,0.872255
2,326.104186,-40.900180,0.814683,24.272838,0.068317
3,326.106237,-40.899279,0.501247,25.474828,0.027796


## Posteriors

## Calculate

In [16]:
P_Ox, P_Ux = Path.calc_posteriors('fixed', box_hwidth=30.)

## Show

In [17]:
Path.candidates

,ra,dec,ang_size,mag,P_O,p_xO,P_Ox
0,326.101738,-40.899771,0.458869,25.295416,0.031633,0.000000e+00,0.000000e+00
1,326.105365,-40.900239,1.308629,21.319569,0.872255,5.156772e-02,9.895195e-01
2,326.104186,-40.900180,0.814683,24.272838,0.068317,6.973468e-03,1.048049e-02
3,326.106237,-40.899279,0.501247,25.474828,0.027796,9.296108e-20,5.684350e-20
